# Not all movies have a foreign release!!~~~!!!!!!

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

%matplotlib inline

In [3]:
# if needed: pip install requests
import requests

url = 'http://boxofficemojo.com/movies/?id=biglebowski.htm'

response = requests.get(url)

In [6]:
response.status_code

200

In [8]:
page_lebowski = response.text
soup_lebowski = BeautifulSoup(page_lebowski)

In [9]:
for element in soup_lebowski.find_all(class_='mp_box_content'):
    print(element, '\n')

(<div class="mp_box_content">\n<table border="0" cellpadding="0" cellspacing="0">\n<tr>\n<td width="40%"><b>Domestic:</b></td>\n<td align="right" width="35%">\xa0<b>$17,451,873</b></td>\n</tr>\n</table>\n</div>, '\n')
(<div class="mp_box_content">\n<table border="0" cellpadding="0" cellspacing="0">\n<tr>\n<td align="center"><a href="/weekend/chart/?yr=1998&amp;wknd=10&amp;p=.htm">Opening\xa0Weekend:</a></td><td>\xa0$5,533,844</td></tr>\n<tr>\n<td align="center" colspan="2"><font size="2">(#6 rank, 1,207 theaters, $4,585 average)</font></td></tr>\n<tr>\n<td align="right">%\xa0of\xa0Total\xa0Gross:</td><td>\xa031.7%</td></tr>\n<tr><td align="right" colspan="2"><font face="Helvetica, Arial, Sans-Serif" size="1"><a href="/movies/?page=weekend&amp;id=biglebowski.htm"><b>&gt; View All 4 Weekends</b></a></font></td></tr>\n</table>\n<table border="0" cellpadding="0" cellspacing="0">\n<tr>\n<td>Widest\xa0Release:</td>\n<td>\xa01,235 theaters</td>\n</tr>\n</table>\n</div>, '\n')
(<div class="mp_

In [10]:
url = 'http://www.boxofficemojo.com/movies/?id=starwars7.htm'

response = requests.get(url)

In [11]:
page_star_wars = response.text
soup_star_wars = BeautifulSoup(page_star_wars)

In [12]:
for element in soup_star_wars.find_all(class_='mp_box_content'):
    print(element,'\n')

(<div class="mp_box_content">\n<table border="0" cellpadding="0" cellspacing="0">\n<tr>\n<td width="40%"><b>Domestic:</b></td>\n<td align="right" width="35%">\xa0<b>$936,662,225</b></td>\n<td align="right" width="25%">\xa0\xa0\xa0<b>45.3%</b></td>\n</tr>\n<tr>\n<td width="40%">+\xa0<a href="/movies/?page=intl&amp;id=starwars7.htm">Foreign:</a></td>\n<td align="right" width="35%">\xa0$1,131,516,000</td>\n<td align="right" width="25%">\xa0\xa0\xa054.7%</td>\n</tr>\n<tr>\n<td colspan="3" width="100%"><hr/></td>\n</tr>\n<tr>\n<td width="40%">=\xa0<b>Worldwide:</b></td>\n<td align="right" width="35%">\xa0<b>$2,068,178,225</b></td>\n<td width="25%">\xa0</td>\n</tr>\n</table>\n</div>, '\n')
(<div class="mp_box_content">\n<table border="0" cellpadding="0" cellspacing="0">\n<tr>\n<td align="center"><a href="/weekend/chart/?yr=2015&amp;wknd=51&amp;p=.htm">Opening\xa0Weekend:</a></td><td>\xa0$247,966,675</td></tr>\n<tr>\n<td align="center" colspan="2"><font size="2">(#1 rank, 4,134 theaters, $59,

In [14]:
import re
foreign_total_regex = re.compile('Foreign Total')
soup_star_wars.find(text=foreign_total_regex)


In [22]:
soup_star_wars.find(text="Foreign:").find_parent("td").find_next_sibling("td").get_text(strip=True)


u'$1,131,516,000'

# use a try except catch 

In [24]:
# for a page that doesnt have a foreign total
soup_lebowski.find(text="Foreign:").find_parent("td").find_next_sibling("td").get_text(strip=True)


AttributeError: 'NoneType' object has no attribute 'find_parent'

In [23]:
soup_star_wars.find(text="Actors:").find_parent("tr").find_all(text=True)[1:]

[]

In [21]:
ftg_string = soup_star_wars.find(text=re.compile('Foreign:'))
print(ftg_string)

AttributeError: 'NavigableString' object has no attribute 'text'

In [20]:
print(ftg_string.findNextSibling())

None
